In [2]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point


In [1]:
print('hi - heat_2')

hi - heat_2


In [3]:
df = pd.read_csv("Data\Phoenix AZ 2023-01-01 to 2024-01-01.csv")

df['datetime'] = pd.to_datetime(df['datetime'])
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day

In [47]:
def EHF (df):
    df['3_day_avg'] = df['temp'].rolling(window=3, min_periods=1).mean().fillna(0)
    q95 = df['temp'].quantile(0.95)
    df['difference_from_95th'] = df['3_day_avg']-q95 
    df['status'] = df['3_day_avg'].apply(lambda x: 'Not Heated' if x <q95 else 'Heated')
    df['30_day_avg'] = df['temp'].rolling(window=30, min_periods=1).mean().shift(1).fillna(0)
    df['3_day_minus_30_day_avg'] = df['3_day_avg'] - df['30_day_avg']
    df['EHF'] = df.apply(lambda x: x['difference_from_95th']* max(1, x['3_day_minus_30_day_avg']) if x['status'] =='Heated' else 0, axis=1 )
    return df 


In [52]:
ariz = EHF(df)
ariz.loc[ariz['EHF']>0][['month','day','temp','EHF']]

In [ ]:
# Iterate over the DataFrame in reverse order to update the status
for i in range(len(df) - 1, -1, -1):
    if df.loc[i, 'updated_status'] == 'Heated':
        if i + 1 < len(df):
            df.loc[i + 1, 'updated_status'] = 'Heated'
        if i + 2 < len(df):
            df.loc[i + 2, 'updated_status'] = 'Heated'



In [ ]:
gr = df.groupby('month').agg({'temp':'mean'})
gr.plot()